In [24]:
%matplotlib inline

In [25]:
import time

import numpy as np

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge
import matplotlib.pyplot as plt

rng = np.random.RandomState(0)


In [26]:
import pandas
#data = pandas.read_csv(filename)
col_list = ["Date/Time", "Wind Speed (m/s)"]
#from google.colab import files 
#uploaded = files.upload()
data = pandas.read_csv("tfgdatos.csv", usecols=col_list)
data

,Date/Time,Wind Speed (m/s)
0,01 01 2018 00:00,5.311336
1,01 01 2018 00:10,5.672167
2,01 01 2018 00:20,5.216037
3,01 01 2018 00:30,5.659674
4,01 01 2018 00:40,5.577941
...,...,...
50525,31 12 2018 23:10,11.404030
50526,31 12 2018 23:20,7.332648
50527,31 12 2018 23:30,8.435358
50528,31 12 2018 23:40,9.421366


In [27]:
np_data = data.to_numpy()
np_data
print(len(np_data))

50530


In [28]:
X = np.empty(shape=5053)
y = np.empty(shape=5053)
   
#d = np.arange(0,500,10)
d = np.arange(0,505300,10)
y = data["Wind Speed (m/s)"].ravel()

from sklearn.model_selection import train_test_split
#y, y_test = train_test_split(aa, test_size = 1, shuffle=False)
X = np.row_stack(d)

X_plot = np.linspace(0, 5, 50530)[:, None]

len(y)

50530

In [ ]:
# #############################################################################
# Fit regression model
train_size = 100
svr = GridSearchCV(
    SVR(kernel="rbf", gamma=0.1),
    param_grid={"C": [1e0, 1e1, 1e2, 1e3], "gamma": np.logspace(-2, 2, 5)},
)

t0 = time.time()
svr.fit(X[:train_size], y[:train_size])
svr_fit = time.time() - t0
print("SVR complexity and bandwidth selected and model fitted in %.3f s" % svr_fit)


sv_ratio = svr.best_estimator_.support_.shape[0] / train_size
print("Support vector ratio: %.3f" % sv_ratio)

t0 = time.time()
y_svr = svr.predict(X_plot)
svr_predict = time.time() - t0
print("SVR prediction for %d inputs in %.3f s" % (X_plot.shape[0], svr_predict))



# #############################################################################
# Look at the results
sv_ind = svr.best_estimator_.support_
plt.scatter(
    X[sv_ind],
    y[sv_ind],
    c="r",
    s=50,
    label="SVR support vectors",
    zorder=2,
    edgecolors=(0, 0, 0),
)
plt.scatter(X[:100], y[:100], c="k", label="data", zorder=1, edgecolors=(0, 0, 0))
plt.plot(
    X_plot,
    y_svr,
    c="r",
    label="SVR (fit: %.3fs, predict: %.3fs)" % (svr_fit, svr_predict),
)

plt.xlabel("Time")
plt.ylabel("Wind Speed")
plt.title("SVR")
plt.legend()

# Visualize training and prediction time
plt.figure()




# Generate sample data
#X = 5 * rng.rand(10000, 1)
#y = np.sin(X).ravel()
#y[::5] += 3 * (0.5 - rng.rand(X.shape[0] // 5))

X = np.empty(shape=5053)
y = np.empty(shape=5053)
   
#d = np.arange(0,500,10)
d = np.arange(0,505300,10)
y = data["Wind Speed (m/s)"].ravel()

from sklearn.model_selection import train_test_split
#y, y_test = train_test_split(aa, test_size = 1, shuffle=False)
X = np.row_stack(d)

sizes = np.logspace(1, 4, 7).astype(int)
for name, estimator in {
   # "KRR": KernelRidge(kernel="rbf", alpha=0.1, gamma=10),
    "SVR": SVR(kernel="rbf", C=1e1, gamma=10),
}.items():
    train_time = []
    test_time = []
    for train_test_size in sizes:
        t0 = time.time()
        estimator.fit(X[:train_test_size], y[:train_test_size])
        train_time.append(time.time() - t0)

        t0 = time.time()
        estimator.predict(X_plot[:10000]) #############
        test_time.append(time.time() - t0)

    plt.plot(
        sizes,
        train_time,
        "o-",
        color="r" if name == "SVR" else "g",
        label="%s (train)" % name,
    )
    plt.plot(
        sizes,
        test_time,
        "o--",
        color="r" if name == "SVR" else "g",
        label="%s (test)" % name,
    )

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Train size")
plt.ylabel("Time (seconds)")
plt.title("Execution Time")
plt.legend(loc="best")

# Visualize learning curves
plt.figure()

svr = SVR(kernel="rbf", C=1e1, gamma=0.1)
#kr = KernelRidge(kernel="rbf", alpha=0.1, gamma=0.1)
train_sizes, train_scores_svr, test_scores_svr = learning_curve(
    svr,
    X[:100],
    y[:100],
    train_sizes=np.linspace(0.1, 1, 10),
    scoring="neg_mean_squared_error",
    cv=10,
)

plt.plot(train_sizes, -test_scores_svr.mean(1), "o-", color="r", label="SVR")
#plt.plot(train_sizes, -test_scores_kr.mean(1), "o-", color="g", label="KRR")
plt.xlabel("Train size")
plt.ylabel("Mean Squared Error")
plt.title("Learning curves")
plt.legend(loc="best")

plt.show()

SVR complexity and bandwidth selected and model fitted in 0.282 s
Support vector ratio: 0.850
SVR prediction for 50530 inputs in 0.748 s
